# Problem description
**Free the Bunny Prisoners**

You need to free the bunny prisoners before Commander Lambda's space station explodes! Unfortunately, the commander was very careful with her highest-value prisoners - they're all held in separate, maximum-security cells. The cells are opened by putting keys into each console, then pressing the open button on each console simultaneously. When the open button is pressed, each key opens its corresponding lock on the cell. So, the union of the keys in all of the consoles must be all of the keys. The scheme may require multiple copies of one key given to different minions.

The consoles are far enough apart that a separate minion is needed for each one. Fortunately, you have already freed some bunnies to aid you - and even better, you were able to steal the keys while you were working as Commander Lambda's assistant. The problem is, you don't know which keys to use at which consoles. The consoles are programmed to know which keys each minion had, to prevent someone from just stealing all of the keys and using them blindly. There are signs by the consoles saying how many minions had some keys for the set of consoles. You suspect that Commander Lambda has a systematic way to decide which keys to give to each minion such that they could use the consoles.

You need to figure out the scheme that Commander Lambda used to distribute the keys. You know how many minions had keys, and how many consoles are by each cell.  You know that Command Lambda wouldn't issue more keys than necessary (beyond what the key distribution scheme requires), and that you need as many bunnies with keys as there are consoles to open the cell.

Given the number of bunnies available and the number of locks required to open a cell, write a function solution(num_buns, num_required) which returns a specification of how to distribute the keys such that any num_required bunnies can open the locks, but no group of (num_required - 1) bunnies can.

Each lock is numbered starting from 0. The keys are numbered the same as the lock they open (so for a duplicate key, the number will repeat, since it opens the same lock). For a given bunny, the keys they get is represented as a sorted list of the numbers for the keys. To cover all of the bunnies, the final answer is represented by a sorted list of each individual bunny's list of keys.  Find the lexicographically least such key distribution - that is, the first bunny should have keys sequentially starting from 0.

num_buns will always be between 1 and 9, and num_required will always be between 0 and 9 (both inclusive).  For example, if you had 3 bunnies and required only 1 of them to open the cell, you would give each bunny the same key such that any of the 3 of them would be able to open it, like so:
[
  [0],
  [0],
  [0],
]
If you had 2 bunnies and required both of them to open the cell, they would receive different keys (otherwise they wouldn't both actually be required), and your answer would be as follows:
[
  [0],
  [1],
]
Finally, if you had 3 bunnies and required 2 of them to open the cell, then any 2 of the 3 bunnies should have all of the keys necessary to open the cell, but no single bunny would be able to do it.  Thus, the answer would be:
[
  [0, 1],
  [0, 2],
  [1, 2],
]

**Test cases**
Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.

-- Python cases --

Input:solution.solution(2, 1)

Output:    [[0], [0]]


Input:solution.solution(4, 4)

Output:    [[0], [1], [2], [3]]


Input:solution.solution(5, 3)

Output:    [[0, 1, 2, 3, 4, 5], [0, 1, 2, 6, 7, 8], [0, 3, 4, 6, 7, 9], [1, 3, 5, 6, 8, 9], [2, 4, 5, 7, 8, 9]]



# Introduction

Let $B$ be the total number of Bunnies and $N$ be the number of required bunnies. Given a set of TK keys (acronym from *TotalKeys*), each key distributed to BPK bunnies (acronym from *buniesPerKey*), we’ll have that:
$$
TK = {B\choose BPK} = \frac{B!}{BPK!*(B-BPK)!}
$$
where ${B\choose BPK}$ denotes the combination of B elements taken BPK at the time \cite{1}.

To find the value of BPK, we can use the Pigeon Hole Principle \cite{2}: for $k,n \in N$,  if $n$ objects are distributed among $m$ sets, then at least one of the sets will contain at least $k+1$ objects, being $k=\frac{n-1}{m}$.

To our specific problem, $k=TK={B\choose BPK}$, $m=BPK$, and $n={B \choose N} N +1$, leading to:
$$
{B\choose BPK} = \frac{{B\choose N}N}{BPK}
$$

$$
(B-BPK)! (BPK-1)! = (B-N)! (N-1)!, \forall BPK \neq N
$$

$$
\therefore BPK = B-N+1
$$

Therefore, to solve this problem optimally we can assign each key to BPK bunnies, from a total of B bunnies, that is:
$$
combinations(range(B),BPK)
$$

# Related works

Alternatively, it is possible to choose ${B-1 \choose B-N}=\frac{B-1!}{(B-N)!*(N-1)!}$ keys to each bunny subject to the constraint that each key is assigned to $BPK$ bunnies, but that requires much more computation. 

Another option is to calculate the total number of keys using the python `itertools.combinations()` tool, instead of using the ${B\choose BPK}$ equation, as was implemented by \cite{3}. This implementation was compared to mine, running the algorithm 10.000 times to the input (5,3) in a computer with Intel Core i5 7200U 2.5GHz processor and 8GB DDR4 of memory.  The results are compiled in the table below. 

| Algorithm            | Runtime |
| -------------------- | ------- |
| Solution unoptimized | 72.2 ms |
| Solution optimized   | 58.9 ms |

# References

[1] - Combination. Wikipedia. Available in: https://en.wikipedia.org/wiki/Combination

[2] - Pigeon Hole Principle. Wikipedia. Available in: https://en.wikipedia.org/wiki/Pigeonhole_principle

[3] - Vasani, Hiren. Github repository. Available in: https://github.com/hirenvasani/foobar/blob/master/free_the_bunny_prisoners.py



In [ ]:
from itertools import combinations

# This solution as developed by \cite{3}
def solutionUnoptimized(num_buns, num_required):
    r = []
    combi = list(combinations(range(num_buns),num_required))
    total = len(combi)*num_required
    repeat_times = (num_buns - num_required) + 1

    new_combi = list(combinations(range(num_buns),repeat_times))
    
    for i in range(num_buns):
        r.append([])
    x = total/repeat_times

    for i in range(int(total/repeat_times)):
        for j in new_combi[i]:
            r[j].append(i)
    return r

def solution(B, N):
    # possible combinations of {B \choose BPK} = {B \choose (B-N+1)} 
    keys2bunnies = list(combinations(range(B),B - N + 1))
    
    # That is just a micro optimization, but is possible to prove that:
    # {B \choose N}*N/BPK = {B \choose BPK}
    # we have already calculated {B \choose BPK}, given that {B \choose BPK} = len(keys2bunnies)
    bunnies2keys = [[] for _ in range(B)]
    for i in range(len(keys2bunnies)):
        for j in keys2bunnies[i]:
            bunnies2keys[j].append(i)
    return bunnies2keys

print (solution(5,3))
#[[0, 1, 2, 3, 4, 5], [0, 1, 2, 6, 7, 8], [0, 3, 4, 6, 7, 9], [1, 3, 5, 6, 8, 9], [2, 4, 5, 7, 8, 9]]

In [ ]:
# Alternative calculation
from math import factorial
factorial(num_buns)/(factorial(num_required)*factorial(num_buns-num_required))

In [ ]:
import time
import gc

gc.collect()
start_time = time.time()
ns = [(5,3)]*10000
res = [solutionUnoptimized(n[0], n[1]) for n in ns]
print("--- %s miliseconds ---" % ((time.time() - start_time)*1000))
#print(res)